In [1]:
from appgeopy import *
from my_packages import *

In [2]:
gps_stations = gpd.read_file(
    r".\Shapefiles\extract_INCangle_NewGPS2023.shp",
    engine="pyogrio",
    read_geometry=False,
)
gps_stations.head(5)

,STATION,X_TWD97,Y_TWD97,INC_ANG
0,ANES,173423.7707,2.622432e+06,34.527428
1,CHSG,177597.8018,2.639739e+06,36.107101
2,CHUA,204982.5358,2.662409e+06,41.966854
3,DNAN,193697.0268,2.619008e+06,38.509434
4,ERLN,190852.6565,2.632728e+06,38.461205


In [3]:
fpath = r"E:\SUBSIDENCE_PROJECT_DATA\GPS_2023_YehDaGang\2023_NEW_20240515\20240527_PROCESSED_DATA\_MERGED\6__Difference_Modeled\Modeled_GPS_Differences_NEU_mm_v1.xlsx"

savefolder = os.path.dirname(fpath)
savename = "Modeled_GPS_Differences_LOS_mm_2022_2023_v1.xlsx"
savepath = os.path.join(savefolder, savename)

available_sheetnames = data_io.get_sheetnames(fpath)

# select_sheet = available_sheetnames[0]
cache = []
for select_sheet in tqdm(available_sheetnames):

    select_gps = gps_stations.query("STATION==@select_sheet")

    if len(select_gps) > 0:
        inc_angle = select_gps.INC_ANG.values[0]

        df = pd.read_excel(
            fpath,
            parse_dates=[0],
            usecols=[0, 4, 5, 6],
            index_col=[0],
            sheet_name=select_sheet,
        )

        LOS_disp = df.apply(
            lambda row: insartools.get_LOS_disp(
                *row, incidence_angle=inc_angle
            ),
            axis=1,
        )

        df["model_LOS(mm)"] = LOS_disp
        df = df.reset_index(drop=False)
        data_io.save_df_to_excel(
            df_to_save=df,
            filepath=savepath,
            sheet_name=select_sheet,
            verbose=False,
        )
    else:
        cache.append(select_sheet)

100%|██████████████████████████████████████████████████████████████████████████████████| 52/52 [00:19<00:00,  2.73it/s]
